<a href="https://colab.research.google.com/github/widyamelia26/Algoritma-Astar-Rute-Destinasi-Wisata-Bali/blob/main/PKA_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from IPython.display import display, HTML, Image
import os

In [20]:
class Graph:
    def __init__(self):
        self.edges = {}  # Dictionary untuk menyimpan edges dan cost antar node

    def add_edge(self, from_node, to_node, cost):
        # Menambahkan edge dari node asal ke node tujuan beserta cost-nya (jarak)
        if from_node not in self.edges:
            self.edges[from_node] = []
        self.edges[from_node].append((to_node, cost))

        # Menambahkan edge dari node tujuan ke node asal (graph tidak berarah)
        if to_node not in self.edges:
            self.edges[to_node] = []
        self.edges[to_node].append((from_node, cost))

    def heuristic(self, node, goal):
        # Koordinat lintang dan bujur untuk node dan goal
        lat1, lon1 = pos[node]
        lat2, lon2 = pos[goal]

        # Konversi derajat ke radian
        lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

        # Haversine formula
        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

        # Radius bumi dalam kilometer
        R = 6371.0
        distance = R * c
        return distance

    def a_star(self, start, goal):
        open_list = []
        heapq.heappush(open_list, (0, start))

        came_from = {}
        g_score = {node: float('inf') for node in self.edges}
        g_score[start] = 0

        f_score = {node: float('inf') for node in self.edges}
        f_score[start] = self.heuristic(start, goal)

        while open_list:
            _, current = heapq.heappop(open_list)

            if current == goal:
                return self.reconstruct_path(came_from, current), g_score[current]  # Mengembalikan jalur dan total jarak

            for neighbor, cost in self.edges[current]:
                tentative_g_score = g_score[current] + cost

                if tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = g_score[neighbor] + self.heuristic(neighbor, goal)
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

        return None, float('inf')  # Jika tidak ada jalur, kembalikan None dan jarak infinity

    def reconstruct_path(self, came_from, current):
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        return total_path[::-1]  # Membalikkan jalur

    def draw_path(self, path):
        G = nx.Graph()
        # Add only nodes and edges that are in the path
        for node in path:
            if node not in G:
                G.add_node(node, pos=pos[node])
        for node1, node2 in zip(path, path[1:]):
            if G.has_edge(node1, node2):
                G[node1][node2]['weight'] = self.edges[node1][self.get_edge_index(node1, node2)][1]
            else:
                G.add_edge(node1, node2, weight=self.get_edge_cost(node1, node2))

        pos_nx = {node: (lat, lon) for node, (lat, lon) in pos.items()}
        labels = nx.get_edge_attributes(G, 'weight')

        plt.figure(figsize=(50, 1000))
        nx.draw(G, pos=pos_nx, with_labels=True, node_size=1000, node_color='lightblue', font_size=7, font_weight='bold', edge_color='grey')
        nx.draw_networkx_edge_labels(G, pos=pos_nx, edge_labels=labels)
        path_edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(G, pos=pos_nx, edgelist=path_edges, edge_color='red', width=2.5)
        plt.title('Graf Rute Terbaik')
        plt.show()

    def get_edge_index(self, node1, node2):
        """Returns the index of the edge between node1 and node2."""
        for i, (neighbor, cost) in enumerate(self.edges[node1]):
            if neighbor == node2:
                return i
        return None

    def get_edge_cost(self, node1, node2):
        """Returns the cost of the edge between node1 and node2."""
        for neighbor, cost in self.edges[node1]:
            if neighbor == node2:
                return cost
        return None



In [3]:
# Membaca data dari file Excel
file_path = 'tourist_destinations.xlsx'

# Membaca sheet untuk destinasi (koordinat)
destinations_df = pd.read_excel(file_path, sheet_name='Sheet1')

destinations_df_renamed = destinations_df.rename(columns={
    'Destination': 'Nama Destinasi',
    'Latitude': 'Garis Lintang',
    'Longitude': 'Garis Bujur'})

print("Tabel Lokasi Destinasi Wisata:")
display(HTML('<div style="overflow-x:auto;">' + destinations_df_renamed.head().to_html(classes='table table-striped', escape=False) + '</div>'))

# Membaca sheet untuk edge (koneksi antar destinasi)
edges_df = pd.read_excel(file_path, sheet_name='Edges')

edges_df_renamed = edges_df.rename(columns={
    'From': 'Dari',
    'To': 'Ke',
    'Cost': 'Jarak (km)'})

print("Tabel Jarak antar Destiansi:")
display(HTML('<div style="overflow-x:auto;">' + edges_df_renamed.head().to_html(classes='table table-striped', escape=False) + '</div>'))

# Membuat graph kosong
g = Graph()

# Inisialisasi posisi node (latitude dan longitude)
pos = {}
for index, row in destinations_df.iterrows():
    pos[row['Destination']] = (row['Longitude'], row['Latitude'])

# Menambahkan edge (koneksi antar destinasi) ke dalam graph dengan jarak
for index, row in edges_df.iterrows():
    g.add_edge(row['From'], row['To'], row['Cost'])

# Menampilkan graph dengan ukuran besar
plt.figure(figsize=(100, 300))  # Mengatur ukuran graph

# Menggambar graph tanpa memperhatikan jarak pada edge
G_nx = nx.Graph()
for index, row in destinations_df.iterrows():
    G_nx.add_node(row['Destination'], pos=(row['Longitude'], row['Latitude']))

for index, row in edges_df.iterrows():
    G_nx.add_edge(row['From'], row['To'], weight=row['Cost'])

pos_nx = nx.get_node_attributes(G_nx, 'pos')
nx.draw(G_nx, pos_nx, with_labels=True, node_size=10000, node_color='lightblue', font_size=20, font_weight='bold')

# Mengambil jarak antar node dan menampilkan sebagai label di atas edge
edge_labels = nx.get_edge_attributes(G_nx, 'weight')
nx.draw_networkx_edge_labels(G_nx, pos, edge_labels=edge_labels)

# Menampilkan graph
plt.title('Rute dari Setiap Destinasi')
plt.show()

Tabel Lokasi Destinasi Wisata:


,Nama Destinasi,Garis Lintang,Garis Bujur
0,pelabuhan gilimanuk,-8.16206,114.43689
1,tanah lot,-8.62114,115.08679
2,bedugul,-8.28301,115.17150
3,sangeh,-8.48002,115.20668
4,tampak siring,-8.41572,115.31508


Tabel Jarak antar Destiansi:


,Dari,Ke,Jarak (km)
0,pelabuhan gilimanuk,tanah lot,117.0
1,pelabuhan gilimanuk,bedugul,104.0
2,tanah lot,pelabuhan gilimanuk,117.0
3,tanah lot,bedugul,48.3
4,tanah lot,sangeh,28.1


In [9]:
# Jalankan A* untuk mencari jalur terpendek
# Daftar start points
start_points = ['pelabuhan gilimanuk', 'bandara ngurahrai']

# Inisialisasi list untuk menyimpan hasil
results = []

# Jalankan A* untuk mencari jalur terpendek
for start in start_points:
    for goal in destinations_df['Destination']:
        if goal != start:
            path, total_km = g.a_star(start, goal)


            # Menambahkan hasil ke dalam list
            results.append({
                'Lokasi Awal': start,
                'Lokasi Tujuan': goal,
                'Jarak Terpendek': ' -> '.join(path),
                'Total Jarak (km)': total_km,
            })

# Membuat DataFrame dari hasil
results_df = pd.DataFrame(results)

# Pengaturan pandas untuk menampilkan seluruh isi kolom tanpa terpotong
pd.set_option('display.max_colwidth', None)

# Menampilkan hasil sebagai tabel yang bisa digulir ke samping
display(HTML(results_df.to_html(classes='table table-striped', escape=False)))

,Lokasi Awal,Lokasi Tujuan,Jarak Terpendek,Total Jarak (km)
0,pelabuhan gilimanuk,tanah lot,pelabuhan gilimanuk -> tanah lot,117.0
1,pelabuhan gilimanuk,bedugul,pelabuhan gilimanuk -> bedugul,104.0
2,pelabuhan gilimanuk,sangeh,pelabuhan gilimanuk -> bedugul -> sangeh,122.9
3,pelabuhan gilimanuk,tampak siring,pelabuhan gilimanuk -> bedugul -> sangeh -> tampak siring,152.7
4,pelabuhan gilimanuk,pantai seminyak,pelabuhan gilimanuk -> tanah lot -> pantai seminyak,134.2
5,pelabuhan gilimanuk,pantai kuta,pelabuhan gilimanuk -> tanah lot -> pantai seminyak -> pantai kuta,137.6
6,pelabuhan gilimanuk,joger,pelabuhan gilimanuk -> tanah lot -> pantai seminyak -> pantai kuta -> joger,139.4
7,pelabuhan gilimanuk,bandara ngurahrai,pelabuhan gilimanuk -> tanah lot -> pantai seminyak -> pantai kuta -> joger -> bandara ngurahrai,142.8
8,pelabuhan gilimanuk,krisna pusat oleh-oleh,pelabuhan gilimanuk -> tanah lot -> pantai seminyak -> pantai kuta -> joger -> krisna pusat oleh-oleh,142.5
9,pelabuhan gilimanuk,bali bird park,pelabuhan gilimanuk -> bedugul -> sangeh -> bali bird park,140.4


In [21]:
# Fungsi untuk menjalankan algoritma A star
def run_a_star(start, goal):
    if start not in pos or goal not in pos:
        return "Error: Lokasi tidak valid!"

    path, total_distance = g.a_star(start, goal)

    if path:
        g.draw_path(path)  # Menampilkan graf dari rute terbaik
        result = f"Jalur terpendek: {' -> '.join(path)}\nTotal jarak: {total_distance:.2f} km"
    else:
        result = "Tidak ada jalur yang ditemukan."

    return result
# Membuat widget input untuk start dan goal
start_dropdown = widgets.Dropdown(
    options=list(pos.keys()),
    description='Lokasi Awal:',
)

goal_dropdown = widgets.Dropdown(
    options=[loc for loc in pos.keys() if loc != start_dropdown.value],
    description='Lokasi Tujuan:',
)

output_label = widgets.Output()

# Update opsi 'goal' berdasarkan pilihan di 'start'
def update_goal_options(change):
    goal_dropdown.options = [loc for loc in pos.keys() if loc != change['new']]

# Event handler untuk dropdown start
start_dropdown.observe(update_goal_options, names='value')

# Fungsi untuk menampilkan hasil
def on_button_click(b):
    with output_label:
        output_label.clear_output()
        start = start_dropdown.value
        goal = goal_dropdown.value
        result = run_a_star(start, goal)
        print(result)

# Tombol untuk menjalankan A star
button = widgets.Button(description="Cari Jalur Terpendek")
button.on_click(on_button_click)

# Menampilkan semua widget
display(start_dropdown, goal_dropdown, button, output_label)


Dropdown(description='Lokasi Awal:', options=('pelabuhan gilimanuk', 'tanah lot', 'bedugul', 'sangeh', 'tampak…

Dropdown(description='Lokasi Tujuan:', options=('tanah lot', 'bedugul', 'sangeh', 'tampak siring', 'pantai sem…

Button(description='Cari Jalur Terpendek', style=ButtonStyle())

Output()